# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
vacation_df = pd.read_csv("../output_data/cities.csv",encoding='utf-8')
vacation_df = vacation_df.iloc[: , 1:]
vacation_df.head()

,Country,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Date
0,BR,Ariquemes,-9.9133,-63.0408,91.42,40,76,2.71,1657549802
1,NZ,Bluff,-46.6000,168.3333,38.80,82,30,14.12,1657549562
2,US,Jamestown,42.0970,-79.2353,77.95,63,0,5.75,1657549562
3,CN,Erenhot,43.6475,111.9767,78.37,35,94,21.34,1657549885
4,CL,Punta arenas,-53.1500,-70.9167,24.91,80,0,14.97,1657549822


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)

heat_layout = {
    'width': '900px',
    'height': '450px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=heat_layout,center=(20.0, 0.0), zoom_level=1)

locations = vacation_df[['Latitude', 'Longitude']]
weights = vacation_df['Humidity']

fig.add_layer(gmaps.heatmap_layer(locations, 
                                  weights=weights,
                                  dissipating=False, 
                                  max_intensity=100, 
                                  point_radius=3))
fig


Figure(layout=FigureLayout(border='2px solid black', height='450px', margin='0 auto 0 auto', padding='2px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
perfect_weather_df = vacation_df.loc[(vacation_df["Cloudiness"] == 0) &
                                      (vacation_df["Max Temp (F)"] > 70) &
                                      (vacation_df["Max Temp (F)"] < 80) &
                                      (vacation_df["Wind Speed"] < 10), :]
perfect_weather_df.dropna(axis='columns')

perfect_weather_df.head()

,Country,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Date
2,US,Jamestown,42.0970,-79.2353,77.95,63,0,5.75,1657549562
95,US,Bethel,41.3712,-73.4140,79.47,66,0,8.05,1657549903
118,FI,Ilmajoki,62.7333,22.5667,72.88,48,0,3.44,1657549907
151,CA,Vernon,50.2581,-119.2691,70.12,61,0,5.75,1657549913
248,MZ,Xai-xai,-25.0519,33.6442,74.66,51,0,3.74,1657549932


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotel_df = perfect_weather_df.loc[:,['Country','City','Latitude','Longitude']]
hotel_df["Hotel"] = ""

hotel_df.head()

,Country,City,Latitude,Longitude,Hotel
2,US,Jamestown,42.0970,-79.2353,
95,US,Bethel,41.3712,-73.4140,
118,FI,Ilmajoki,62.7333,22.5667,
151,CA,Vernon,50.2581,-119.2691,
248,MZ,Xai-xai,-25.0519,33.6442,


In [51]:
google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
     "radius": 5000, 
     "keyword":"hotel",
     "key": g_key,
}


In [52]:
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    long = row["Longitude"]
    city_name = row["City"]

    params["location"] = f"{lat},{long}"

    response = requests.get(base_url, params=params).json()

    results = response["results"]
    
    try:
        hotel_df.loc[index,"Hotel"] = results[0]['name']
              
    except (KeyError, IndexError):
        hotel_df.loc[index,"Hotel"] = "No close hotels in the area"

hotel_df

,Country,City,Latitude,Longitude,Hotel
2,US,Jamestown,42.0970,-79.2353,Chautauqua Harbor Hotel
95,US,Bethel,41.3712,-73.4140,La Quinta Inn & Suites by Wyndham Danbury
118,FI,Ilmajoki,62.7333,22.5667,Hotelli-Kahvila Kuppi
151,CA,Vernon,50.2581,-119.2691,"Holiday Inn Express & Suites Vernon, an IHG Hotel"
248,MZ,Xai-xai,-25.0519,33.6442,No close hotels in the area
295,US,Barstow,34.8986,-117.0228,Best Western Desert Villa Inn
485,US,Lathrop,37.8227,-121.2766,Hampton Inn & Suites Manteca


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
